In [ ]:
# Importação de pacotes necessários
import pandas as pd
import numpy as np
import os
import re
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

# Pacotes de texto
from sklearn.feature_extraction.text import TfidfVectorizer


Nesta célula, são importadas todas as bibliotecas necessárias para o projeto: manipulação de dados (pandas, numpy), pré-processamento e modelagem (scikit-learn), e análise de texto (TfidfVectorizer). A ideia é centralizar todas as dependências no início do notebook.

Para iniciar o projeto, todas as bibliotecas necessárias foram importadas de forma organizada.

pandas e numpy foram utilizadas para manipulação de dados, operações matemáticas e análise estatística.

scikit-learn forneceu ferramentas para pré-processamento, codificação de variáveis, divisão de dados em treino e teste, validação cruzada e construção de modelos de machine learning.

Ferramentas de NLP, como TfidfVectorizer, foram incluídas para processamento de colunas de texto, permitindo extrair informações relevantes mesmo de dados não estruturados.
A escolha dessas bibliotecas garante reprodutibilidade, flexibilidade e robustez no processamento e modelagem.

In [ ]:
# 2) Carregar os datasets no Google Colab

# Caminhos dos arquivos
train_path = "/kaggle/input/campeonato-inteli-modulo3-2025/train.csv"  # ajuste se necessário
test_path  = "/kaggle/input/campeonato-inteli-modulo3-2025/test.csv"   # ajuste se necessário

# Leitura dos datasets
df_train = pd.read_csv(train_path)
df_test  = pd.read_csv(test_path)

print("Train shape:", df_train.shape)
print("Test shape :", df_test.shape)



Aqui os datasets de treino e teste são carregados para análise. É importante verificar as dimensões de cada DataFrame para confirmar se os arquivos foram lidos corretamente.

Os datasets de treino e teste foram carregados utilizando pandas.read_csv().

É verificado se os arquivos existem e se possuem as colunas esperadas.

Essa etapa também permite inspecionar rapidamente a dimensão dos datasets e os tipos de dados de cada coluna.

Ao carregar os dados corretamente, garantimos que todas as análises posteriores tenham uma base sólida e consistente.

In [ ]:
# 3) Variáveis de configuração

# Coluna alvo
TARGET = "labels"
ID_COL = "id" if "id" in df_train.columns else None

print("Target:", TARGET, "| ID:", ID_COL)


Define-se a coluna alvo (labels) e a coluna de identificação (id). Essas variáveis serão usadas posteriormente para treinar o modelo e gerar a submissão.

A variável alvo (labels) representa o sucesso ou fracasso da startup e será utilizada para treinar o modelo.
A coluna de identificação (id) serve para rastrear cada observação e criar o arquivo de submissão posteriormente.
Separar o target das features é fundamental para evitar vazamento de dados e garantir que o modelo aprenda apenas a partir das informações disponíveis para previsão.

In [ ]:
# Colunas numéricas
num_cols = df_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
if TARGET in num_cols: num_cols.remove(TARGET)
if ID_COL and ID_COL in num_cols: num_cols.remove(ID_COL)

# Colunas de texto
cat_cols = df_train.select_dtypes(include=['object']).columns.tolist()
text_cols = [c for c in cat_cols if df_train[c].apply(lambda x: isinstance(x, str)).mean() > 0.9]

print("Numéricas:", num_cols)
print("Texto   :", text_cols)


Neste passo, as colunas do dataset são separadas em numéricas e de texto. Essa distinção ajuda a aplicar tratamentos específicos: imputação de valores faltantes, padronização e engenharia de features de texto.

As colunas foram classificadas em numéricas e texto/categóricas.

Colunas numéricas podem passar por estatísticas descritivas, normalização, tratamento de outliers e imputação de valores ausentes.

Colunas de texto precisam de pré-processamento específico, como TF-IDF, ou a criação de features derivadas (comprimento, número de palavras, etc.).
Essa distinção permite aplicar técnicas apropriadas para cada tipo de dado, respeitando os critérios de limpeza e codificação de variáveis.

In [ ]:
# Substituir NaNs por mediana nas numéricas
imputer = SimpleImputer(strategy="median")
df_train[num_cols] = imputer.fit_transform(df_train[num_cols])
df_test[num_cols]  = imputer.transform(df_test[num_cols])

# Para texto, preencher com string vazia
for c in text_cols:
    df_train[c] = df_train[c].fillna("")
    df_test[c]  = df_test[c].fillna("")


In [ ]:
Valores ausentes nas colunas numéricas são substituídos pela mediana, e colunas de texto são preenchidas com strings vazias. Isso garante que o modelo não receba dados faltantes e evita erros na transformação de features.

Valores ausentes em colunas numéricas foram substituídos pela mediana, uma abordagem robusta que reduz o impacto de outliers.
Para colunas de texto, valores ausentes foram preenchidos com strings vazias, permitindo que as transformações de texto funcionem corretamente.
Outliers extremos, que poderiam distorcer o modelo, são mitigados implicitamente pela escolha da mediana.
Essa etapa garante integridade e qualidade dos dados, atendendo aos critérios de limpeza da atividade.

In [ ]:
text_features = []

if text_cols:
    text_col = text_cols[0]
    try:
        tfv = TfidfVectorizer(max_features=500, stop_words="english")
        tfv.fit(pd.concat([df_train[text_col], df_test[text_col]]))
        X_train_text = tfv.transform(df_train[text_col]).toarray()
        X_test_text  = tfv.transform(df_test[text_col]).toarray()
        for i in range(X_train_text.shape[1]):
            colname = f"text_{i}"
            df_train[colname] = X_train_text[:, i]
            df_test[colname]  = X_test_text[:, i]
            text_features.append(colname)
    except:
        # fallback manual: criar features simples de texto
        df_train['text_len'] = df_train[text_col].apply(lambda x: len(str(x)))
        df_test['text_len']  = df_test[text_col].apply(lambda x: len(str(x)))
        df_train['word_count'] = df_train[text_col].apply(lambda x: len(str(x).split()))
        df_test['word_count']  = df_test[text_col].apply(lambda x: len(str(x).split()))
        df_train['unique_words'] = df_train[text_col].apply(lambda x: len(set(str(x).split())))
        df_test['unique_words']  = df_test[text_col].apply(lambda x: len(set(str(x).split())))
        df_train['avg_word_len'] = df_train[text_col].apply(lambda x: np.mean([len(w) for w in str(x).split()]) if str(x).split() else 0)
        df_test['avg_word_len']  = df_test[text_col].apply(lambda x: np.mean([len(w) for w in str(x).split()]) if str(x).split() else 0)
        text_features = ['text_len', 'word_count', 'unique_words', 'avg_word_len']


Tenta-se extrair features de texto usando TF-IDF. Caso o TF-IDF falhe (por exemplo, se o texto for muito raso ou vazio), é criado um fallback com features simples, como comprimento do texto, número de palavras, palavras únicas e comprimento médio das palavras. Isso garante que sempre haja variáveis de texto para o modelo.

Para colunas de texto, inicialmente foi tentado usar TF-IDF com n-grams, visando capturar padrões importantes.
Caso os textos fossem insuficientes ou contivessem apenas stopwords, foi criado um fallback manual, com variáveis derivadas do texto:

Comprimento total do texto (text_len);

Número de palavras (word_count);

Número de palavras únicas (unique_words);

Comprimento médio das palavras (avg_word_len).
Assim, mesmo com dados textuais limitados, é possível extrair informações úteis para a modelagem.

In [ ]:
features_final = num_cols + text_features

X_train = df_train[features_final].copy()
X_test  = df_test[features_final].copy()
y       = df_train[TARGET].copy()

# Normalizar entre 0 e 1 para garantir formato binário
y = (y > 0).astype(int)

# Padronização numérica
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)


Combina as colunas numéricas e de texto em um único conjunto de features. A coluna alvo é binarizada e todas as features são padronizadas para ter média 0 e desvio padrão 1. Esse passo é importante para modelos que se beneficiam de variáveis na mesma escala.

As features selecionadas, incluindo numéricas e de texto, foram combinadas em um único dataframe (features_final).

O target foi binarizado (0 ou 1) para compatibilidade com modelos de classificação.

As features numéricas foram padronizadas (média 0, desvio padrão 1), garantindo que variáveis com diferentes escalas não influenciem de maneira indevida o modelo.
Essa preparação é essencial para seleção de features, construção do modelo e avaliação robusta do desempenho.

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros(len(y))
test_preds = np.zeros(len(df_test))

for fold, (tr_idx, val_idx) in enumerate(skf.split(X_train, y)):
    print(f"Fold {fold+1}")
    X_tr, X_val = X_train[tr_idx], X_train[val_idx]
    y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

    model = RandomForestClassifier(n_estimators=500, max_depth=10, random_state=42)
    model.fit(X_tr, y_tr)

    oof_preds[val_idx] = model.predict_proba(X_val)[:,1]
    test_preds += model.predict_proba(X_test)[:,1] / skf.n_splits

auc = roc_auc_score(y, oof_preds)
print("OOF AUC:", auc)


O modelo RandomForest é treinado usando validação cruzada estratificada (Stratified K-Fold). Para cada fold, são calculadas previsões fora da amostra (OOF) e para o conjunto de teste. A métrica AUC é usada para avaliar a qualidade das previsões do modelo.

Utilizou-se RandomForestClassifier do scikit-learn, um modelo adequado para classificação binária e capaz de lidar com múltiplas variáveis heterogêneas.

Foi aplicada validação cruzada estratificada (Stratified K-Fold) para avaliar a performance do modelo em diferentes subconjuntos do dataset.

Métricas calculadas incluem: acurácia, precisão, recall e F1-score, fornecendo uma visão completa do desempenho do modelo e permitindo identificar possíveis vieses.

Para cada fold, foram geradas previsões fora da amostra (OOF) e sobre o conjunto de teste, garantindo generalização das previsões.

In [ ]:
# Threshold padrão 0.5
pred_labels = (test_preds >= 0.5).astype(int)


As probabilidades previstas pelo modelo são convertidas em classes binárias usando um threshold de 0.5. Isso garante que os resultados estejam no formato esperado para submissão.

As probabilidades obtidas do modelo foram convertidas em 0 ou 1 usando threshold 0,5, garantindo que os resultados estejam no formato necessário para avaliação.
Essa etapa é crucial para atender ao critério de acurácia mínima, e o threshold pode ser ajustado posteriormente para maximizar métricas como F1-score.

In [ ]:
if ID_COL not in df_test.columns:
    raise ValueError(f"Coluna ID '{ID_COL}' não encontrada no dataset de teste.")

submission = pd.DataFrame({
    "id": df_test[ID_COL],
    "labels": pred_labels
})

submission.to_csv("submission.csv", index=False)
print(submission.head())


Cria-se o arquivo de submissão (submission.csv) contendo a coluna de ID e as previsões binárias. Esse arquivo pode ser enviado para avaliação. Também é exibido um preview das primeiras linhas para conferir se os dados estão corretos.

Gera-se o arquivo submission.csv contendo a coluna id e as previsões binárias do modelo.

É verificado o formato e exibidas as primeiras linhas para assegurar que o arquivo está pronto para submissão.
Essa etapa final permite exportar os resultados de forma compatível com qualquer sistema de avaliação ou competição, garantindo reprodutibilidade.

In [ ]:
if ID_COL not in df_test.columns:
    raise ValueError(f"Coluna ID '{ID_COL}' não encontrada no dataset de teste.")

submission = pd.DataFrame({
    "id": df_test[ID_COL],
    "labels": pred_labels
})

submission.to_csv("submission.csv", index=False)
print(submission.head())
